## Movie Recommendations based on Ratings

With Collaborative filtering we make predictions (filtering) about the interests of a user by collecting preferences or taste information from many users (collaborating). The underlying assumption is that if a user A has the same opinion as a user B on an issue, A is more likely to have B's opinion on a different issue x than to have the opinion on x of a user chosen randomly.

The image below (from Wikipedia) shows an example of collaborative filtering. At first, people rate different items (like videos, images, games). Then, the system makes predictions about a user's rating for an item not rated yet. The new predictions are built upon the existing ratings of other users with similar ratings with the active user. In the image, the system predicts that the user will not like the video.

<img src=https://upload.wikimedia.org/wikipedia/commons/5/52/Collaborative_filtering.gif />

Spark MLlib library for Machine Learning provides a Collaborative Filtering implementation by using __Alternating Least Squares (ALS)__. The implementation in MLlib has these parameters:

* numBlocks is the number of blocks used to parallelize computation (set to -1 to auto-configure).
* rank is the number of latent factors in the model.
* iterations is the number of iterations to run.
* lambda specifies the regularization parameter in ALS.
* implicitPrefs specifies whether to use the explicit feedback ALS variant or one adapted for implicit feedback data.
* alpha is a parameter applicable to the implicit feedback variant of ALS that governs the baseline confidence in preference observations.


In [21]:
# Initialize pyspark
import findspark
findspark.init()
import pyspark

In [22]:
# Initialize and create ba spark session
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('movie').getOrCreate()

In [23]:
# Using Spark to read the movie lens data set.
data = spark.read.csv('movielens_ratings.csv', header=True, inferSchema=True)

In [24]:
# Printing the first few rows of the dataframe
data.show(4)

+-------+------+------+
|movieId|rating|userId|
+-------+------+------+
|      2|   3.0|     0|
|      3|   1.0|     0|
|      5|   2.0|     0|
|      9|   4.0|     0|
+-------+------+------+
only showing top 4 rows



In [25]:
data.describe().show()

+-------+------------------+------------------+------------------+
|summary|           movieId|            rating|            userId|
+-------+------------------+------------------+------------------+
|  count|              1501|              1501|              1501|
|   mean| 49.40572951365756|1.7741505662891406|14.383744170552964|
| stddev|28.937034065088994| 1.187276166124803| 8.591040424293272|
|    min|                 0|               1.0|                 0|
|    max|                99|               5.0|                29|
+-------+------------------+------------------+------------------+



We can do a split to evaluate how well our model performed, but it is very hard to know conclusively how well a recommender system is truly working for some topics. Especially if subjectivity is involved, for example not everyone that loves star wars is going to love star trek, even though a recommendation system may suggest otherwise.

In [26]:
#Since the dataset is smaller, we can use 80% and 20% for train-test splits

train_data,test_data = data.randomSplit([0.8,0.2])

In [27]:
train_data.describe().show()

+-------+------------------+------------------+------------------+
|summary|           movieId|            rating|            userId|
+-------+------------------+------------------+------------------+
|  count|              1201|              1201|              1201|
|   mean| 49.79766860949209| 1.765195670274771|14.378018318068277|
| stddev|28.714947232410005|1.1774921015366675| 8.543144409583485|
|    min|                 0|               1.0|                 0|
|    max|                99|               5.0|                29|
+-------+------------------+------------------+------------------+



In [28]:
test_data.describe().show()

+-------+------------------+-----------------+------------------+
|summary|           movieId|           rating|            userId|
+-------+------------------+-----------------+------------------+
|  count|               300|              300|               300|
|   mean|47.836666666666666|             1.81|14.406666666666666|
| stddev|29.806903459910927|1.227022935515444| 8.794644666210528|
|    min|                 0|              1.0|                 0|
|    max|                99|              5.0|                29|
+-------+------------------+-----------------+------------------+



In [29]:
# Setting up Recommender system
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.recommendation import ALS

In [30]:
# Build the recommendation model using ALS on the training data
als = ALS(userCol='userId', itemCol='movieId', ratingCol='rating')

In [31]:
model = als.fit(train_data)

Now let's see how the model performed!

In [32]:
predictions = model.transform(data)

In [33]:
predictions.show(4)

+-------+------+------+----------+
|movieId|rating|userId|prediction|
+-------+------+------+----------+
|     31|   1.0|    26| 1.0442168|
|     31|   1.0|    27| 1.2237741|
|     31|   4.0|    12| 3.0750914|
|     31|   1.0|    13| 0.9127061|
+-------+------+------+----------+
only showing top 4 rows



Evaluating the model by computing the RMSE on the test data


In [34]:
evaluator = RegressionEvaluator(predictionCol='prediction', labelCol='rating', metricName='rmse')

In [35]:
rmse = evaluator.evaluate(predictions)

In [36]:
print("Root Mean Squared Error:",rmse)

Root Mean Squared Error: 0.584323339262094


**The RMSE 0.58 described our error in terms of the stars rating column.**

## Recommendation for a particular user

In [37]:
single_user = test_data.select('userId','movieId').filter("userId == 11")

In [38]:
single_user.show()

+------+-------+
|userId|movieId|
+------+-------+
|    11|      6|
|    11|     13|
|    11|     18|
|    11|     20|
|    11|     23|
|    11|     25|
|    11|     35|
|    11|     38|
|    11|     41|
|    11|     45|
|    11|     47|
|    11|     48|
|    11|     64|
|    11|     67|
|    11|     77|
|    11|     94|
|    11|     97|
+------+-------+



In [39]:
recommendations = model.transform(single_user)

In [40]:
recommendations.orderBy('prediction', ascending=False).show()

+------+-------+----------+
|userId|movieId|prediction|
+------+-------+----------+
|    11|     18|  4.384326|
|    11|     35| 2.9970126|
|    11|     94| 2.6027384|
|    11|     64| 2.5451334|
|    11|     13| 2.5430214|
|    11|     23|  1.861339|
|    11|     77| 1.7974586|
|    11|     20| 1.7660612|
|    11|     38| 1.7064192|
|    11|      6| 1.4461564|
|    11|     48| 1.4300591|
|    11|     41| 1.3201851|
|    11|     47| 1.2331718|
|    11|     97| 1.1224972|
|    11|     67| 1.0781276|
|    11|     45| 1.0287994|
|    11|     25|0.63714236|
+------+-------+----------+



In [ ]:
#Closing the spark session
spark.stop()